In [3]:
from collections import Counter                        #####IMPORTSRequired1#####
import itertools
import math
from collections import defaultdict
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import operator
import time

/home/kapil/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kapil/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kapil/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package stopwords to /home/kapil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#doc=f.read() ############Required2############
start=time.time()
def text_processing(line1):
    text=line1.split(' \t')[1].rpartition("> ")[2]
    text= " ".join(text.split("."))
    text=text.lower()
    string=[]
    for words in text.split():
        words = ("".join(e for e in words if e.isalpha()))
        if not words in stop_words:
            string.append(ps.stem(words))
    return string
end=time.time()
#print(a)
print(end-start)
#print(message.split('@en .\n')[1].split(' \t')[1].rpartition("> ")[2])    

0.00020265579223632812


In [5]:
stop_words = set(stopwords.words('english')) ########Required3#######
ps = PorterStemmer()

In [6]:

Ind_doc=defaultdict(list)                         ############THIS BLOCK CREATES LIST OF LISTRequired4##############
big_doc=defaultdict(list)
entire_corpus=[]
i=0
N_doc=0
labels=[]
start1=time.time()
with open('verysmall_train.txt') as f:
    lines=f.readlines()
    lines=lines[3:]
    N_doc+=1
    for line in lines:
        labels.append(line.split(' \t')[0].split(','))
        
        for ctgr in (line.split(' \t')[0].split(",")):
            big_doc[ctgr].append(text_processing(line))
            entire_corpus+=text_processing(line)
end1=time.time()        
print(end1-start1)

62.82851767539978


In [7]:
C=Counter(list(itertools.chain.from_iterable(labels)))######Required5#######

In [ ]:
print(C)

In [8]:
V=len(set(entire_corpus)) #######Required6#######
print(V)      
if 'power' in list(set(entire_corpus)):
    print("yes")


53058
yes


In [9]:
############THIS BLOCK MERGES LIST OF LIST INTO SINGLE LISTRequired7##############
set1=set(list(itertools.chain.from_iterable(labels)))
bigdoc=defaultdict(list)
print(len(set1))
#print(len(Ind_doc[3].split(",")))
for ctgr in (list(set1)):
    for t in range(len(big_doc[ctgr])):
        bigdoc[ctgr]+=big_doc[ctgr][t]
#print(Counter(bigdoc['Articles_containing_video_clips']))
#print(list(big_doc['Articles_containing_video_clips']))
#print(list(big_doc['Articles_containing_video_clips'][2]+big_doc['Articles_containing_video_clips'][3]))

49


In [10]:
##########THIS GIVES PRIOR LOG FREQUENCIES FOR WHOLE CORPUSRequired8#############
logprior=defaultdict(int)
#logprior=math.log10(int(C.values())/N)
for key, value in C.items():
    prob=math.log10(value/N_doc)
    logprior[key]=prob
#print(logprior)

In [11]:
def nested_dict(n, type): #######Required9#######
    if n == 1:
        return defaultdict(type)
    else:
        return defaultdict(lambda: nested_dict(n-1, type))

In [12]:
count = nested_dict(2, list)        ########Required10#########
loglikelihood = nested_dict(2, list)
ctgr_count=defaultdict(list)
for ctgr in list(set1):
    g=Counter(bigdoc[ctgr])
    ctgr_count[ctgr]=0
    for word in g.keys():
        count[ctgr][word]=g[word]
        ctgr_count[ctgr]+=g[word]
    for word in g.keys():      
        loglikelihood[ctgr][word]=math.log10((count[ctgr][word]+1)/(ctgr_count[ctgr]+V))
#print(count['Articles_containing_video_clips']['anim'])
#print(loglikelihood['Articles_containing_video_clips']['anim'])
#print(ctgr_count['Articles_containing_video_clips'])

In [35]:
sum_ctgr=defaultdict(float) ##########Required11#########
success=0
failure=0
st=time.time()
with open('verysmall_train.txt') as f1:
    lines=f1.readlines()
    for line in lines[:20]:
        text_for_test=text_processing(line)
        for ctgr in list(set1):
            sum_ctgr[ctgr]=logprior[ctgr]
            for word in list(text_for_test):
                if word in list(set(entire_corpus)):
                    sum_ctgr[ctgr]=sum_ctgr[ctgr]+int(loglikelihood[ctgr].get(word,'0'))
        prediction=min(sum_ctgr.items(), key=operator.itemgetter(1))[0]
        if prediction in list(line.split(' \t')[0].split(',')):
            success+=1
        else:
            failure+=1
en=time.time()
print(en - st)        
    

11614.933624267578


In [40]:
print(success/(success+failure))

0.85


In [37]:
print(failure)

3


In [34]:
print(sorted(sum_ctgr))
print(prediction)
print(min(sum_ctgr.items(), key=operator.itemgetter(1))[0])

['American_comedy_films', 'American_drama_films', 'American_film_actresses', 'American_film_directors', 'American_films', 'American_male_film_actors', 'American_male_television_actors', 'American_military_personnel_of_World_War_II', 'American_people_of_Irish_descent', 'American_television_actresses', 'Articles_containing_video_clips', 'Association_football_defenders', 'Association_football_forwards', 'Association_football_goalkeepers', 'Association_football_midfielders', 'Asteroids_named_for_people', 'Australian_rules_footballers_from_Victoria_(Australia)', 'Black-and-white_films', 'Brazilian_footballers', 'British_films', 'Columbia_University_alumni', 'Deaths_from_myocardial_infarction', 'English-language_albums', 'English-language_films', 'English-language_journals', 'English-language_television_programming', 'English_cricketers', 'English_footballers', 'Fellows_of_the_Royal_Society', 'French_films', 'German_footballers', 'Guggenheim_Fellows', 'Harvard_University_alumni', 'Hindi-lang

In [31]:
with open('verysmall_test.txt') as f1:
    lines=f1.readlines()
    print(text_processing(line))

['apollo', '', 'spaceflight', 'land', 'first', 'human', 'moon', 'american', 'neil', 'armstrong', 'buzz', 'aldrin', 'juli', '', '', '', 'utc', 'armstrong', 'becam', 'first', 'step', 'onto', 'lunar', 'surfac', 'six', 'hour', 'later', 'juli', '', '', 'utc', 'armstrong', 'spent', 'two', 'half', 'hour', 'outsid', 'spacecraft', 'aldrin', 'slightli', 'less', 'togeth', 'collect', '', '', 'pound', '', '', 'kg', 'lunar', 'materi', 'return', 'earth', 'third', 'member', 'mission', 'michael', 'collin', 'pilot', 'command', 'spacecraft', 'alon', 'lunar', 'orbit', 'armstrong', 'aldrin', 'return', 'day', 'later', 'trip', 'back', 'earth', 'launch', 'saturn', 'v', 'rocket', 'kennedi', 'space', 'center', 'merritt', 'island', 'florida', 'juli', '', 'apollo', '', 'fifth', 'man', 'mission', 'nasa', 'apollo', 'program', 'apollo', 'spacecraft', 'three', 'part', 'command', 'modul', 'cm', 'cabin', 'three', 'astronaut', 'part', 'land', 'back', 'earth', 'servic', 'modul', 'sm', 'support', 'command', 'modul', 'prop

In [ ]:
f = open('verysmall_train.txt','r')                ######EXTRACT LABELS###### 
message = f.read()
N=10495
while(i!=N):
    labels.append(message.split('@en .\n')[i].split(' \t')[0].split(','))
    i+=1


In [ ]:
Ind_doc=defaultdict(list)                         ############THIS BLOCK CREATES LIST OF LIST##############
big_doc=defaultdict(list)
entire_corpus=[]
i=0
while(i!=N):
    Ind_doc[i]=message.split('@en .\n')[i].split(' \t')[0]
    for ctgr in (Ind_doc[i].split(",")):
        big_doc[ctgr].append(text_processing(message,i))
        entire_corpus+=text_processing(message,i)
    i+=1

#print("--------------------------------")
#print(logprior.keys())

In [ ]:
a=text_processing(message,2)
#print(C)
sum=0
g=Counter(bigdoc['Articles_containing_video_clips'])
print(g['anim'])
for k,v in g.items():
#    print(k,v)
    sum+=v
#file = open("testfile.txt","w") 
#file.write(str(Counter(bigdoc['Articles_containing_video_clips'])))

In [ ]:
print(sum)

In [ ]:
f1 = open('verysmall_test.txt','r')
test_text=f1.read()

In [ ]:


for ctgr in list(set1):
    sum_ctgr[ctgr]=logprior[ctgr]
    text_for_test=text_processing(test_text,k)
    for word in list(text_for_test):
        if word in list(set(entire_corpus)):
            sum_ctgr[ctgr]=sum_ctgr[ctgr]+int(loglikelihood[ctgr].get(word,'0'))


In [ ]:
print(test_text.split('@en .\n')[6].split(' \t')[1].rpartition("> ")[2])
print(test_text.split('@en .\n')[6].split(' \t')[0])

In [ ]:
print(loglikelihood['Indian_films'].get('clay',0))
print(type(loglikelihood['Indian_films'].get('clay',0)))
print(sum_ctgr['English-language_television_programming'])
print(min(sum_ctgr.items(), key=operator.itemgetter(1))[0])
print(sum_ctgr)

In [ ]:
print(set1)

In [ ]:
text_for_test=text_processing(test_text,5)
print(text_for_test)
print(sorted(sum_ctgr.items()))